In [ ]:
%matplotlib notebook

import netCDF4
import numpy as np
import matplotlib.tri as tri
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib
from matplotlib import cm

def make_map(projection=ccrs.PlateCarree()):
    
    fig, ax = plt.subplots(figsize=(6, 6),
                           subplot_kw=dict(projection=projection))
#     gl = ax.gridlines(draw_labels=True)
#     gl.xlabels_top = gl.ylabels_right = False
#     gl.xformatter = LONGITUDE_FORMATTER
#     gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

def plot_gtiff(ax, data, transform, cmap='viridis', vmin=None, vmax=None, cbar=False, cbar_kwargs={}, **kwargs):
    extent = plot_extent(data, transform)
    im = ax.imshow(data, extent=extent, vmin=vmin, vmax=vmax, cmap=cmap, origin='upper', **kwargs)
    ax.set_extent(extent)
    
    dmin, dmax = np.nanmin(data), np.nanmax(data)
    vmin = vmin if vmin is not None else dmin 
    vmax = vmax if vmax is not None else dmax
    if (dmax > vmax) and (dmin < vmin):
        cbar_kwargs.update(extend='both')
    elif dmax > vmax:
        cbar_kwargs.update(extend='max')
    elif dmin < vmin:
        cbar_kwargs.update(extend='min')

    if cbar:
        cbar = set_colorbar(ax, im, **cbar_kwargs)
    else:
        cbar = None
    return im, cbar

def plot_extent(data, transform):
    rows, cols = data.shape[-2:]
    left, top = transform * (0, 0)
    right, bottom = transform * (cols, rows)
    extent = (left, right, bottom, top)
    return extent

In [ ]:
import glofrim.glofrim_lib as glib 
from configparser import ConfigParser
from os.path import join, isfile, abspath, dirname, basename, normpath

In [ ]:
from glofrim.glofrim_lib import configread
env_fn = r'../../environment.env'
config = configread(env_fn)
testDataDir = config.get('general', 'test_dir')

In [ ]:
_cf = ConfigParser(inline_comment_prefixes=('#'))
env = glib.configread(abspath(env_fn), encoding='utf-8', cf=_cf)
glib.config2dict(env)

In [ ]:
# config_fn = join(testDataDir, 'glofrim_PCR2CMF.ini')
# config_fn = join(testDataDir, 'glofrim_PCR2DFM.ini')
config_fn = join(testDataDir, 'glofrim_WFL2DFM.ini')
# config_fn = join(testDataDir, 'glofrim_PCR2LFP.ini')
config_fn

In [ ]:
from glofrim import Glofrim
cbmi = Glofrim()
cbmi.initialize_config(config_fn, env_fn=env_fn)
cbmi.exchanges

In [ ]:
# config_fn = r'/home/dirk/repos/model_test_data/test_Elbe/WFL_Elbe/SBM/wflow_sbm.ini'
# cbmi.bmimodels['WFL']._bmi.initialize_config(config_fn)
# cbmi.bmimodels['WFL']._bmi.dynModel.clonemappath

In [ ]:
cbmi.initialize_model()

In [ ]:
import numpy as np
c1 = cbmi.exchanges[2][1]['coupling'] 
c1.set_frac()
c1.to_ind.shape, c1.from_ind.shape, c1.frac.shape

In [ ]:
# fcoords = c1.to_bmi.grid.get_poly_coords()
# fpatches = (matplotlib.patches.Polygon(face) for face in fcoords)
# fpc = matplotlib.collections.PatchCollection(fpatches, edgecolor='blue')
# fpc.set_facecolor('none')

In [ ]:
fig, ax = make_map()
# grid cells
cells = c1.from_bmi.grid.get_poly_coords()
gpatches = (matplotlib.patches.Polygon(cell) for cell in cells)
gpc = matplotlib.collections.PatchCollection(gpatches, edgecolor='grey')
gpc.set_facecolor('none')
# color linked cells
g = np.ma.masked_equal(np.zeros(c1.from_bmi.grid.shape), 0)
g.flat[c1.from_ind] = c1.from_grp
# all nodes
n = c1.to_bmi.grid._1d.nodes
ax.add_collection(gpc)
# ax.add_collection(fpc)
plot_gtiff(ax, g, c1.from_bmi.grid.transform, cmap=cm.Set1)
ax.scatter(n[:,0], n[:,1], c='grey', s=0.1)
ax.scatter(n[c1.to_ind,0], n[c1.to_ind,1], c=c1.to_grp, s=2, cmap=cm.Set1)

In [ ]:
cbmi.get_start_time(), cbmi.get_current_time(), cbmi.get_end_time(), cbmi.get_time_step()

In [ ]:
from datetime import datetime
cbmi.update_until(datetime(2000, 1, 10))
cbmi.get_current_time()

In [ ]:
# cbmi.finalize()